In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import h5py
import numpy as np
import statistics 

In [ ]:
def get_reads_dict(filename):
    file = h5py.File(filename, "r")
    file = file['Reads']
    reads = []
    for r in file.keys():
        elem = {}
        elem['UUID'] = r
        for k in file[r].keys():
            elem[k]=file[r][k][()]
        reads.append(elem)
    return reads

def normalise_list(lst):
    mmin = min(lst)
    mmax = max(lst)
    lst = (lst - mmin)/(mmax - mmin)
    return lst

def are_signals_equal(s1,s2):
    if(len(s1) != len(s2)):
        return False

    are_equal = True
    for idx, val in enumerate(s1):
        if(s1[idx] != s2[idx]):
            return False
    return True

def find_matches_in_tayiaki_ouput(mapped_reads, unmapped_reads):
    matched = []
    unmatched = []

    for r1 in mapped_reads:
        s1 = r1['Dacs']
        found = None
        for r2 in unmapped_reads:
            s2 = r2['Dacs']
            if(are_signals_equal(s1,s2)):
                found = r2
        if(found != None):
            matched.append([r1,found])
        else:
            unmatched.append(r1)
    
    return matched, unmatched

def load_reads(f1,f2):
    unmapped_reads = get_reads_dict(f1)
    mapped_reads = get_reads_dict(f2)
    
    return find_matches_in_tayiaki_ouput(mapped_reads, unmapped_reads)

def get_read_attributes(read):
    #id, signal, points, labels
    return read['UUID'], normalise_list(read['Dacs']), read['Ref_to_signal'], read['Reference']

In [ ]:
f1 = "./../../taiyakiOutputs/output_createfasta.hdf5"
f2 = "./../../taiyakiOutputs/output_justfromfasta.hdf5"

reads, other_reads = load_reads(f1,f2)

In [ ]:
print(len(reads))
print(len(other_reads))

In [ ]:
index = 1

_, signal1, points1, labels1 = get_read_attributes(reads[index][0])
_, signal2, points2, labels2 = get_read_attributes(reads[index][1])

plt.figure(figsize=(30,5))
plt.plot(signal1[:2000], 'r', label="mapped")
plt.show()

plt.figure(figsize=(30,5))
plt.plot(signal2[:2000], 'b', label="not mapped")
plt.show()

In [ ]:
def make_dna_list(read):
    dnalist = []
    pos = 0
    signalrefs = list(read['Ref_to_signal'])
    dnaref = list(read['Reference'])
    for idx in range(len(signalrefs)):
        if idx == 0:
            dnalist.extend([-1]*signalrefs[idx])
        else:
            for i in range(signalrefs[idx]-signalrefs[idx-1]):
                dnalist.append(dnaref[idx-1])
    return dnalist

def make_labels(dna_unmapped, dna_mapped):
    DNA = []
    
    i = 0
    while(i < len(dna_mapped)):
        
        #mapped have readins
        if(dna_mapped[i] != -1):
            DNA.append(dna_mapped[i])
        #mapped dont yet have readings
        else:
            #unmapped have readings
            if(dna_unmapped[i] != -1):
                DNA.append(-1)
            #unmapped have no readings
            else:
                DNA.append(-2)
        i +=1
    
    return DNA    

In [ ]:
for index in range(len(reads)):
    #THESE TWO HAVE DIF LENGTHS?!
    dna1 = make_dna_list(reads[index][0])
    dna2 = make_dna_list(reads[index][1])
    DNA = make_labels(dna1, dna2)
    uid1, signal1, points1, labels1 = get_read_attributes(reads[index][0])
    _, signal2, points2, labels2 = get_read_attributes(reads[index][1]) 
             
    #plt.figure(figsize=(30,5))
    #plt.plot(signal1[0:2000]*4, 'k', label="raw")
    #plt.plot(dna1[0:2000], 'r', label="base")
    #plt.show()
    
    #plt.figure(figsize=(30,5))
    #plt.plot(signal1[0:2000]*4, 'k', label="raw")
    #plt.plot(dna2[0:2000], 'b', label="base")
    #plt.show()

    print(uid1)
    plt.figure(figsize=(30,5))
    plt.plot(signal1[:2000]*4, 'k', label="raw")
    plt.plot(DNA[:2000], 'g', label="DNA")
    plt.show()